### 作業目的: 熟練自定義collate_fn與sampler進行資料讀取

本此作業主要會使用[IMDB](http://ai.stanford.edu/~amaas/data/sentiment/)資料集利用Pytorch的Dataset與DataLoader進行
客製化資料讀取。
下載後的資料有分成train與test，因為這份作業目的在讀取資料，所以我們取用train部分來進行練習。
(請同學先行至IMDB下載資料)

### 載入套件

In [1]:
# Import torch and other required modules
import glob
import torch
import re
import nltk
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_svmlight_file
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

nltk.download('stopwords') #下載stopwords
nltk.download('punkt') #下載word_tokenize需要的corpus

[nltk_data] Downloading package stopwords to
[nltk_data]     D:\Users\ntuhuser\Anaconda3\envs\tensorflow\nltk_data.
[nltk_data]     ..
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     D:\Users\ntuhuser\Anaconda3\envs\tensorflow\nltk_data.
[nltk_data]     ..
[nltk_data]   Package punkt is already up-to-date!


True

### 探索資料與資料前處理
這份作業我們使用test資料中的pos與neg


In [2]:
# 讀取字典，這份字典為review內所有出現的字詞
with open('./aclImdb/imdb.vocab', 'r', encoding='utf-8') as f:
    vocab = f.read().split('\n')

# 以nltk stopwords移除贅字，過多的贅字無法提供有用的訊息，也可能影響模型的訓練
print(f"vocab length before removing stopwords: {len(vocab)}")
vocab = [word for word in vocab if word not in stopwords]
print(f"vocab length after removing stopwords: {len(vocab)}")

# 將字典轉換成dictionary
vocab_dic = {}
idx = 0
for w in vocab:
    if w not in vocab_dic:
        vocab_dic[w] = idx
        idx += 1
# vocab_dic

vocab length before removing stopwords: 89527
vocab length after removing stopwords: 89356


In [3]:
# 將資料打包成(x, y)配對，其中x為review的檔案路徑，y為正評(1)或負評(0)
# 這裡將x以檔案路徑代表的原因是讓同學練習不一次將資料全讀取進來，若電腦記憶體夠大(所有資料檔案沒有很大)
# 可以將資料全一次讀取，可以減少在訓練時I/O時間，增加訓練速度

review_pairs = []
for file_pos in glob.glob(f'.\\aclImdb\\train\\pos\\*.txt'):
    review_pairs.append((file_pos, 1))
for file_neg in glob.glob(f'.\\aclImdb\\train\\neg\\*.txt'):
    review_pairs.append((file_neg, 0))
print(review_pairs[:2])
print(f"Total reviews: {len(review_pairs)}")

[('.\\aclImdb\\train\\pos\\0_9.txt', 1), ('.\\aclImdb\\train\\pos\\10000_8.txt', 1)]
Total reviews: 25000


### 建立Dataset, DataLoader, Sampler與Collate_fn讀取資料
這裡我們會需要兩個helper functions，其中一個是讀取資料與清洗資料的函式(load_review)，另外一個是生成詞向量函式
(generate_vec)，注意這裡我們用來產生詞向量的方法是單純將文字tokenize(為了使產生的文本長度不同，而不使用BoW)

In [4]:
def load_review(review_path):
    
    with open(review_path, 'r', encoding='utf-8') as f:
        review = f.read()
        
    #移除non-alphabet符號、贅字與tokenize
    review = re.sub('\W', ' ', review)
    review = nltk.word_tokenize(review)
    review = [word for word in review if word not in stopwords]
    return review

def generate_vec(review, vocab_dic):
    bag_vector = np.zeros(len(vocab_dic))
    for word in review:
        if vocab_dic.get(word):
            bag_vector[vocab_dic.get(word)] += 1
            
    return bag_vector

In [5]:
#建立客製化dataset

class dataset(Dataset):
    '''custom dataset to load reviews and labels
    Parameters
    ----------
    data_pairs: list
        directory of all review-label pairs
    vocab: list
        list of vocabularies
    '''
    def __init__(self, data_dirs, vocab):
        self.data_dirs = data_dirs
        self.vocab = vocab

    def __len__(self):
        return len(self.data_dirs)

    def __getitem__(self, idx):
        review_path, tag = self.data_dirs[idx]
        review = load_review(review_path)
        bag_vector = generate_vec(review, self.vocab)
        return (bag_vector, tag)
    

#建立客製化collate_fn，將長度不一的文本pad 0 變成相同長度
def collate_fn(batch):

    reviews, labels = zip(*batch) 
    
    ### create pads for reviews ###
    lengths = [len(x) for x in reviews]
    max_length = max(lengths)
    
    batch_reviews = []
    
    for i in range(len(reviews)):
        # pad reviews
        tmp_pads = torch.zeros(max_length)
        tmp_pads[:lengths[i]] = torch.from_numpy(reviews[i])
        batch_reviews.append(tmp_pads.view(1,-1))

    return torch.cat(batch_reviews,dim=0), torch.tensor(labels) , torch.tensor(lengths)

In [6]:
# 使用Pytorch的RandomSampler來進行indice讀取並建立dataloader
from torch.utils.data import Sampler, SequentialSampler, RandomSampler
custom_dataset = dataset(review_pairs, vocab_dic)
custom_dataloader = DataLoader(custom_dataset,sampler=RandomSampler(custom_dataset),collate_fn=collate_fn,batch_size=2)
next(iter(custom_dataloader))

(tensor([[0., 1., 2.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]]),
 tensor([0, 0]),
 tensor([89356, 89356]))